In [ ]:
!pip install pycoingecko requests tiktoken cohere openai langchain langchain_experimental google-search-results

###1.IMPORT LIBRARY

In [ ]:
import os
import requests
import json
from datetime import datetime

from langchain.document_loaders.base import Document
from langchain.utilities import ApifyWrapper
from langchain.vectorstores import Vectara
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains import RetrievalQA
from langchain.schema.output_parser import StrOutputParser
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate,ChatPromptTemplate, MessagesPlaceholder

###2.KEY SETTINGS

In [ ]:
os.environ["OPENAI_API_KEY"]      = "OPENAI_API_KEY"
os.environ["VECTARA_CUSTOMER_ID"] = "VECTARA_CUSTOMER_ID"
os.environ["VECTARA_CORPUS_ID"]   = "VECTARA_CORPUS_ID"
os.environ["VECTARA_API_KEY"]     = "VECTARA_API_KEY"
os.environ["GOOGLE_API_KEY"]      = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"]       = "GOOGLE_CSE_ID"

###3.INITIALIZE VECTARA

In [ ]:
vectara = Vectara(
    vectara_customer_id = os.getenv("VECTARA_CUSTOMER_ID"),
    vectara_corpus_id   = os.getenv("VECTARA_CORPUS_ID"),
    vectara_api_key     = os.getenv("VECTARA_API_KEY")
)

In [ ]:
# ChatGPT 모델 지정
#llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)
llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

In [ ]:
# Create Prompt
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
{context}
Question: {question}
Relevant text, if any:"""
#prompt = ChatPromptTemplate.from_template(question_prompt_template)
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

#chain_type_kwargs = {"verbose": True,  "question_prompt": QUESTION_PROMPT}

In [ ]:
# Define retriever
retriever = vectara.as_retriever(lambda_val=0.025, k=10, filter=None)
#retriever = vectara.as_retriever()

from langchain.chains import RetrievalQAWithSourcesChain

knowledgeBase_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    verbose=True,
    chain_type_kwargs={
        "prompt": QUESTION_PROMPT
    },
)

In [ ]:
def run_qa_chain(query):
    results = knowledgeBase_qa({"query": query},return_only_outputs=True)
    return str(results)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain import LLMMathChain

# Search
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name='Knowledge Base',
        func=run_qa_chain,
        return_direct=True,
        description=(f'''
            use this tool when answering general knowledge queries to get
            more information about the topic
            '''
        )
    )

]

In [ ]:
agent_template = """You are an expert at extracting information from your knowledge base.
When searching for the first time, a search is performed in the knowledge base,
and if there are insufficient or no results, a Google search is performed and the results are displayed.
Be sure to answer in Korean!
{memory}
Human: {human_input}
Chatbot:"""

agent_prompt = PromptTemplate(input_variables=["memory", "human_input"],template=agent_template)

agent_memory = ConversationBufferMemory(memory_key="memory",prompt=agent_prompt, return_messages=True)

agent_kwargs = {
            "extra_prompt_messages": [MessagesPlaceholder(variable_name="agent_memory")],
        }


In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=agent_memory,
    max_execution_time=1,
    early_stopping_method="generate",
)

In [ ]:
response = agent(
    {
        "input": "2024년 반도체 시장의 전망과 예측"
    }
)

In [ ]:
response['output']

###result 뽑는거....

In [ ]:
def extract_result():

  import re

  # 정규식 패턴
  result_pattern = r"'result': '(.*?)'"

  # 정규식을 사용하여 추출
  result_match = re.search(result_pattern, response['output'])

  # 결과 출력
  if result_match:
      result_value = result_match.group(1)
      #print(result_value)

      return result_value
  else:
      return response['output']

# 결과...
## LLM Result
## Metadata

In [ ]:
source_documents = retriever.get_relevant_documents(
    "2024년 반도체 시장의 전망과 예측"
)

In [ ]:
print(extract_result())

for i in range(len(source_documents)):
  #print(source_documents[i].page_content[0:100])
  print(source_documents[i].metadata)

#안넘어가서 테스트해봄

In [ ]:
"""
We then provide a deep dive on the four main components.
## Document Loaders
How to load documents from a variety of sources.
## Text Splitters
An overview of the abstractions and implementions around splitting text.
## VectorStores
An overview of VectorStores and the many integrations LangChain provides.
## Retrievers
An overview of Retrievers and the implementations LangChain provides.
"""

# Document Loaders
## Using directory loader to load all .md files in a directory
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./sample_data/', glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()
print(len(docs))



In [ ]:
# Text Splitters
from langchain.text_splitter import MarkdownTextSplitter
markdown_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=100, length_function=len)
md_docs = markdown_splitter.split_documents(docs)

# Embeddings

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()



In [ ]:
!pip install faiss-cpu

In [ ]:
# Vector stores (pip install faiss or pip install faiss-cpu)
from langchain.vectorstores import FAISS
db = FAISS.from_documents(md_docs, embeddings)



In [ ]:
# Retrievers
query = "How do I deploy chains?"
retriever = db.as_retriever(search_kwargs={"k": 4})
filtered_docs = retriever.get_relevant_documents(query)
print(len(filtered_docs))



In [ ]:
filtered_docs

In [ ]:
# LLM
from langchain.prompts import PromptTemplate
#from custom_conversation_chain import CustomLLM
from langchain.chains import RetrievalQAWithSourcesChain
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
{context}
Question: {question}
Relevant text, if any:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: 28-pl
QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia's Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: 0-pl
Content: And we won't stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet's use this moment to reset. Let's stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet's stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can't change how divided we've been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who'd grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: 24-pl
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I've always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I'm taking robust action to make sure the pain of our sanctions  is targeted at Russia's economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what's happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
Source: 5-pl
Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt's based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer's, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
Source: 34-pl
=========
FINAL ANSWER: The president did not mention Michael Jackson.
SOURCES:
QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)




In [ ]:
#chain_type_kwargs = {"verbose": True, "combine_prompt": COMBINE_PROMPT, "question_prompt": QUESTION_PROMPT}

chain_type_kwargs = {"verbose": True,  "question_prompt": QUESTION_PROMPT}

qa = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="map_reduce",
                                retriever=retriever, return_source_documents=True,
                                chain_type_kwargs=chain_type_kwargs)
response = qa({"question": "How do i deploy chains?", "verbose": True})
print(response.keys())
print(response["answer"])
print(response["sources"])

In [ ]:
print(response.keys())

In [ ]:
print(response['source_documents'])